<a href="https://colab.research.google.com/github/taeilkeemm/fordatastudy/blob/main/%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

In [53]:
date = str(datetime.now()) 
date = date[:date.rfind(':')].replace(' ', '_') 
date = date.replace(':','시') + '분' 

In [54]:
query = input('검색 키워드를 입력하세요 : ') 
query = query.replace(' ', '+') 

news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : ')) 

검색 키워드를 입력하세요 : 주식
총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : 100


In [55]:
news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

req = requests.get(news_url.format(query))
soup = BeautifulSoup(req.text, 'html.parser')

In [56]:
news_dict = {} 
idx = 0 
cur_page = 1

In [57]:
print()
print('크롤링 중...')

while idx < news_num:
    
    table = soup.find('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
    
    for n in a_list[:min(len(a_list), news_num-idx)]:
        news_dict[idx] = {'title' : n.get('title'),
                          'url' : n.get('href') }
        idx += 1

    cur_page += 1
    
    pages = soup.find('div', {'class' : 'sc_page_inner'})
    next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')
    
    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')


크롤링 중...


In [58]:
print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_{}_{}.xlsx'.format(query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}'.format(folder_path, xlsx_file_name))
##os.startfile(folder_path)

크롤링 완료
데이터프레임 변환
엑셀 저장 완료 | 경로 : /content\네이버뉴스_주식_2022-03-27_10시42분.xlsx


In [59]:
df = news_df

In [60]:
df

,title,url
0,"바이든, 억만장자세 공식 도입…주식 미실현 이익에 과세",http://news.kmib.co.kr/article/view.asp?arcid=...
1,내 주식 휴지조각 될라…‘상폐 주의보 비상’,https://www.sedaily.com/NewsView/263KVWOLHX
2,'주식보다 낫다' MZ세대 우르르…요즘 뜨는 재테크 [차은지의 리치리치],https://www.hankyung.com/finance/article/20220...
3,"돈 빌리며 주식담보 제공…대법 ""갚을 의사, 사기 아냐""",http://www.newsis.com/view/?id=NISX20220324_00...
4,"보령제약, 주식회사 '보령'으로 사명 변경",http://news.mk.co.kr/newsRead.php?no=273871&ye...
...,...,...
95,[소영주의 장외주식 상식투자] 증권플러스 비상장·거울거래소 비상장 '조건부 승인',https://news.mtn.co.kr/news-detail/20220324175...
96,‘살아난 반도체 대장’ 엔비디아 9%↑ [3분 미국주식],http://news.kmib.co.kr/article/view.asp?arcid=...
97,'밈 주식'의 부활…게임스톱 급등,https://www.hankyung.com/finance/article/20220...
98,"모더나 CEO, 코로나19로 떼돈…주식팔아 4900억원",http://www.newsis.com/view/?id=NISX20220318_00...


In [61]:
! pip install konlpy

In [62]:
from konlpy.tag import Okt
from collections import Counter
import csv

In [63]:
#크롤링한 기사 갯수 확인 후 숫자 조정
title = ''
  
for i in range(0, 1000):
  title = title + ' ' + df.loc[i][0]

In [64]:
title

' 바이든, 억만장자세 공식 도입…주식 미실현 이익에 과세 내 주식 휴지조각 될라…‘상폐 주의보 비상’ \'주식보다 낫다\' MZ세대 우르르…요즘 뜨는 재테크 [차은지의 리치리치] 돈 빌리며 주식담보 제공…대법 "갚을 의사, 사기 아냐" 보령제약, 주식회사 \'보령\'으로 사명 변경 보령제약, 주식회사 ‘보령’으로 사명 변경 마크로젠 임원진, 7억원 규모 자사주식 매수 검찰, 고수익 미끼로 무등록 주식프로그램 판매 일당 기소 BNK 금융지주 주식 보유자 역대 최대 배당금 받는다 "주가만 봐도 어질어질해요"...다시 켜진 밈주식 투자 주의보 [추적자추기자] 러 중앙은행 "모스크바 거래소, 월요일부터 주식 채권 거래 정상화" 지난달 주식 발행 87% ‘뚝’…LG엔솔 IPO 기저효과 탓 “러시아 때문에” 中 주식·채권, ‘패닉 셀링’에 자금 유출 거셌다 대신증권, \'국내주식 온라인 거래 수수료 할인\' 이벤트 검찰, 고수익 미끼로 무등록 주식프로그램 판매 일당 기소 [뉴욕증시]美 주식 과매도 왔나…금리 폭등에도 버티는 투심 삼성증권 미국주식 주간거래액 한달새 5000억원 "인재 유출 막아라"…애플, 최대 \'2억\' 주식보너스 또 쐈다 한달만에 부분 재개장 러 주식시장, 소폭 상승 마감 주식양도세, 폐지가 능사 아냐…부과기준 상향 등 절충안 필요 비보존헬스케어, 비보존 주식 15.3% 추가 취득해 최대 주주 올라 박용만 삼부자 두산 주식 전량 매각…두산그룹과 결별 \'전참시\' 박재범 "돈 필요하지만 사람이 더 중요…주식보다 나에게 투자" "조정장 언제 끝날까" 키움증권, 미국주식 투자 반상회 개최 러시아, 24일부터 주식시장 일부 개장…공매도 금지(종합) 무신사 한문일 대표, 주당 200만원 공짜 주식 안 받은 이유 선학개미? 비상장 주식투자, 이렇게 해서 망했다 [코주부] 무신사 창업주, 임직원에 주식 1000억 쏜다 네이버 최수연 대표·김남선 CFO, 회사 주식 1억여원씩 매입 오디오·주식중개·중고패션에 \'뭉칫돈\' 몰린다…투자행렬 안철수, 주식 시장선 이

In [65]:
okt = Okt()
noun = okt.nouns(title)
for i, v in enumerate(noun):
  if len(v) < 2:
    noun.pop(1)

In [66]:
count = Counter(noun)

In [67]:
count

Counter({'가격': 3,
         '가구': 2,
         '가상': 1,
         '가치': 3,
         '감사원': 1,
         '강원': 1,
         '개미': 3,
         '개인': 1,
         '개장': 2,
         '개최': 1,
         '거': 1,
         '거래': 9,
         '거래소': 1,
         '거울': 1,
         '검찰': 1,
         '게임스톱': 1,
         '견인': 1,
         '결별': 1,
         '결정': 1,
         '결합': 1,
         '경제': 1,
         '계약': 1,
         '계절': 1,
         '곳': 2,
         '공매도': 2,
         '공모': 2,
         '공무원': 1,
         '공시': 1,
         '공약': 1,
         '공짜': 1,
         '과': 1,
         '과세': 1,
         '관련': 1,
         '관리': 1,
         '관심': 1,
         '구글': 1,
         '국내': 5,
         '국민': 1,
         '군인공제회': 1,
         '규모': 2,
         '규제': 1,
         '그룹': 2,
         '극복': 1,
         '금': 1,
         '금리': 2,
         '금물': 1,
         '금액': 3,
         '금지': 3,
         '급등': 5,
         '급여': 1,
         '기금': 1,
         '기부': 1,
         '기소': 2,
         '기존': 1,
         '기준': 1,
     

In [68]:
noun_list = count.most_common(400)
for v in noun_list:
  print(v)

('주식', 75)
('증권', 13)
('미국', 10)
('거래', 9)
('투자', 9)
('삼성', 8)
('주식시장', 6)
('비보', 6)
('주', 6)
('직원', 6)
('국내', 5)
('비상', 5)
('급등', 5)
('러시아', 4)
('최대', 4)
('존', 4)
('취득', 4)
('주주', 4)
('돌파', 4)
('증여', 4)
('주식회사', 4)
('인', 4)
('전자', 4)
('발행', 3)
('자금', 3)
('주간', 3)
('애플', 3)
('달', 3)
('폐지', 3)
('등', 3)
('추가', 3)
('두산', 3)
('금지', 3)
('대표', 3)
('개미', 3)
('임', 3)
('유안', 3)
('해외', 3)
('가치', 3)
('반도체', 3)
('가격', 3)
('금액', 3)
('이상', 3)
('인상', 3)
('올해', 3)
('처분', 3)
('뚝', 2)
('효과', 2)
('탓', 2)
('채권', 2)
('유출', 2)
('무', 2)
('등록', 2)
('프로그램', 2)
('기소', 2)
('뉴욕증시', 2)
('매도', 2)
('금리', 2)
('또', 2)
('상승', 2)
('마감', 2)
('주식양도세', 2)
('안', 2)
('존헬', 2)
('스케', 2)
('박용만', 2)
('매각', 2)
('그룹', 2)
('전', 2)
('돈', 2)
('사람', 2)
('더', 2)
('나', 2)
('일부', 2)
('개장', 2)
('공매도', 2)
('일', 2)
('회사', 2)
('매입', 2)
('안랩', 2)
('양도소득세', 2)
('이부진', 2)
('이서현', 2)
('무상', 2)
('시설', 2)
('부동산', 2)
('일가', 2)
('담보', 2)
('종', 2)
('공모', 2)
('규모', 2)
('앤', 2)
('무상증자', 2)
('투자자', 2)
('증권사', 2)
('곳', 2)
('매수', 2)
('가구', 2)
('즉석밥', 2)


In [69]:
! pip install wordcloud

In [70]:
import sys
from wordcloud import WordCloud

In [71]:
#폰트이름 확인!
wc = WordCloud(font_path='SCDream5.otf', background_color="white", width=1000,
               height=1000, max_words=100,
               max_font_size=300)
image = wc.generate_from_frequencies(dict(noun_list))
image.to_file('image2.png')